<span style="display: inline-block; margin-right: 10px; vertical-align: middle; border: 1px solid #F8F8F8; border-radius: 8px; padding: 5px; background-color: #F8F8F8;">
    <a href="https://www.designsafe-ci.org/workspace/jupyter-lab-hpc-cuda-ds?appVersion=1.1.1" target="_parent" style="text-decoration: none;">
        <span style="font-family: Helvetica, sans-serif; font-size: 13px; color: #565656; margin-right: 5px; vertical-align: middle; font-weight: 600;">Open in</span>
        <img src="https://www.designsafe-ci.org/media/filer_public/2d/d3/2dd37fbf-289e-49cf-9c1a-879c864c4e17/nsf_nheri-ds.png" alt="Open in DesignSafe" style="width: 100px; height: auto; vertical-align: middle;">
    </a>
</span>

<span style="display: inline-block; margin-right: 10px; vertical-align: middle;">
    <a href="https://colab.research.google.com/github/NHERI-SimCenter/BrailsPlusPlus/blob/master/examples/image_processor/garage_detector.ipynb" target="_parent">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="width: 130px; height: auto; vertical-align: middle;"/>
    </a>
</span>

<span style="display: inline-block; vertical-align: middle; margin-top: 6px;">
    <a target="_blank" href="https://lightning.ai/new?repo_url=https%3A//github.com/NHERI-SimCenter/BrailsPlusPlus/blob/master/examples/image_processor/garage_detector.ipynb">
        <img src="https://pl-bolts-doc-images.s3.us-east-2.amazonaws.com/app-2/studio-badge.svg" alt="Open in Studio" style="width: 120px; height: auto; vertical-align: middle;"/>
    </a>
</span>

## **Install BRAILS++**
BRAILS++ is not yet available on PyPi. The following line installs the latest version from the GitHub repository using `pip`.

In [1]:
!pip install git+https://github.com/NHERI-SimCenter/BrailsPlusPlus

  Cloning https://github.com/bacetiner/BrailsPlusPlus to /tmp/pip-req-build-0bfpjwiw
  Running command git clone --filter=blob:none --quiet https://github.com/bacetiner/BrailsPlusPlus /tmp/pip-req-build-0bfpjwiw
  Resolved https://github.com/bacetiner/BrailsPlusPlus to commit 7280af6926637fda5cb802c8e2adb3b14558012a
  Preparing metadata (setup.py) ... done
  Created wheel for brails: filename=brails-4.0-py3-none-any.whl size=2008520 sha256=17532d90a4e1d348444b199de0a672b41550eef630fc244d91fc08d6ca15cea8
  Stored in directory: /tmp/pip-ephem-wheel-cache-xda4kwtz/wheels/65/0b/aa/a9113e0557b48ca6dfe52e54ff2df5af4aa9945a3736b53797
Successfully built brails


## **Import ImageSet and Importer Methods of BRAILS++**
BRAILS++ offers various Artificial Intelligence-powered modules for predicting building and infrastructure attributes from images, such as the `GarageDetector` demonstrated in this example.

The `Importer` class provides the main methods to load these modules. To gather the images needed for attribute prediction, users can use the ImageSet class or BRAILS++ scrapers, which facilitate automated asset geometry and image retrieval. This example illustrates both methods for sourcing imagery for use with the `GarageDetector` module.

In [2]:
from brails.types.image_set import ImageSet
from brails import Importer

## **Define Google API Key**
You need a Google API Key with Street View Static API enabled to download the Google Street View imagery required for this example. Please follow this [link](https://support.google.com/googleapi/answer/6158862?hl=en) for instructions on setting up a Google API key.

In [3]:
API_KEY = 'YOUR-API-KEY-HERE'

## **Load Street-Level Imagery: (Option 1) Download Imagery by Specifying the Region Name**
The `GarageDetector` class is created to identify the existence of garages in buildings from street-level imagery. In this option, BRAILS++ retrieves the required imagery by following the steps below.
1. <strong>Call the `RegionBoundary` class with the necessary details to specify the region of interest.</strong> In this example, street-level imagery of buildings in Atlantic County, NJ, are downloaded. Therefore, the required information includes the `type` of information provided and the `data` itself, which are the `locationName` and `Atlantic County, NJ`.

2. <strong>Download the footprint inventory for the `RegionBoundary` saved in `region_boundary_object` by downloading the OpenStreetMap footprint data through the `USA_FootprintScraper`.</strong> The only information required to initialize `USA_FootprintScraper` is the desired output unit for `length`, which, in this case, is `ft`. The output of `USA_FootprintScraper` is an `AssetInventory` saved in `atlantic_fp`.

3. <strong>Get a randomly selected `100` building subset of the footprint inventory obtained using `atlantic_fp`. </strong> This subset is obtained using the `get_random_sample` method of an `AssetInventory` object and is stored in `atlantic_fp_subset`. For this example, the random seed is set to `10` for reproducibility of the results.

4. <strong>Get the street-level imagery for the selected subset using `get_images` method of `GoogleStreetview` module.</strong> `get_images` requires two inputs 1) AssetInventory for which the images will be retrieved (in this example, `atlantic_fp_subset`) and 2) the path to the folder where the retrieved images will be saved, which in this case is `tmp/street/`.


In [4]:
# Select a region and create its RegionBoundary:
importer = Importer()

region_data = {"type": "locationName",
               "data": "Atlantic County, NJ"}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

usa_fp_class = importer.get_class("USA_FootprintScraper")
usa_fp_data = {"length": "ft"}
footprint_scraper = usa_fp_class(usa_fp_data)
atlantic_fp = footprint_scraper.get_footprints(region_boundary_object)

# Subsample from the extracted assets to keep the image downloading step quick.
# Here, we are randomly sampling 100 buildings using a random seed value of 40:
atlantic_fp_subset = atlantic_fp.get_random_sample(100, 10)

# Get street-level imagery for the selected subset using GoogleStreetview:
google_street_class = importer.get_class("GoogleStreetview")
google_street = google_street_class({'apiKey': API_KEY})
images_street = google_street.get_images(
    atlantic_fp_subset, "tmp/street/")


Searching for Atlantic County, NJ...
Found Atlantic County, New Jersey, United States

Meshing the defined area...


Obtaining the number of buildings in each cell: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 344/344 [00:05<00:00, 58.83it/s]



Meshing complete. Split Atlantic County into 385 cells


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 385/385 [00:00<00:00, 71231.01it/s]



Found a total of 93932 building footprints in Atlantic County


INFO:brails.scrapers.google_streetview.google_streetview:
Images will be saved to: /home/bacetiner/Documents/BrailsPlusPlus/examples/image_processor/tmp/street

Obtaining street-level imagery:   1%|█▏                                                                                                                       | 1/100 [00:00<00:11,  8.41it/s]INFO:brails.scrapers.google_streetview.google_streetview:No street-level imagery found for the building located at 39.5190, -74.5928
INFO:brails.scrapers.google_streetview.google_streetview:No street-level imagery found for the building located at 39.4333, -74.6320
INFO:brails.scrapers.google_streetview.google_streetview:No street-level imagery found for the building located at 39.3833, -74.3995
INFO:brails.scrapers.google_streetview.google_streetview:No street-level imagery found for the building located at 39.4350, -74.4879
INFO:brails.scrapers.google_streetview.google_streetview:No street-level imagery found for the building located at 39

## **Predict Existence of Garages in Buildings Using the GarageDetector Module**
`GarageDetector` includes a pre-trained EfficientDet_D4 model that has been trained on a custom dataset of 22,600 images. This model is capable of identifying if garages exist in buildings.

In [5]:
importer = Importer()
garage_detector_class = importer.get_class('GarageDetector')
garage_detector_classifier = garage_detector_class()
predictions = garage_detector_classifier.predict(images_street)


Checking the existence of garages for each building...
Default garage detector model in tmp/models/efficientdet-d4_garageDetector.pth loaded

Performing garage detections...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:09<00:00, 10.51it/s]


Total execution time: 00:00:09.91


## **Show Prediction Results**

In [6]:
print(predictions)

{60110: 1, 75954: 0, 68678: 0, 4174: 0, 32150: 1, 37018: 0, 87104: 0, 83505: None, 87073: 1, 27044: None, 16826: 0, 45664: 0, 36322: 1, 57771: 0, 3752: 0, 69139: 0, 17022: None, 7773: None, 84052: None, 26474: 0, 81209: 0, 92683: 0, 60380: 0, 6969: 0, 23141: None, 79265: None, 41053: None, 14890: 1, 32828: None, 42605: 0, 18980: 0, 67246: 0, 89232: 0, 68601: 0, 12788: 0, 53396: 0, 6433: 0, 41387: None, 44031: 0, 29762: None, 20161: 0, 40659: 0, 21312: None, 68208: 0, 40467: 0, 91159: None, 38193: 0, 60531: 0, 20706: 0, 76822: None, 27585: 0, 64486: 0, 56808: None, 67676: 1, 8440: 0, 22387: 1, 76509: None, 26867: 0, 79859: None, 76719: 0, 1173: 1, 11613: 1, 61218: 0, 80132: 1, 39744: None, 14714: 0, 25050: 0, 31574: 0, 60469: None, 59466: None, 13013: 0, 61786: None, 83690: None, 43486: 0, 81497: 0, 59399: 0, 37358: 0, 69904: None, 75087: None, 32379: None, 9152: 0, 4867: 1, 19487: None, 79589: 1, 50446: 0, 24490: 0, 58513: 0, 14140: 1, 54527: None, 24858: None, 76968: 0, 2531: 0, 54802